# ChatBot using Embeddings

## Ceating Model Outline

In [ ]:
# imports
import json
import os
import random
import time
from tqdm import tqdm
import pickle
from scipy.spatial.distance import cosine
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

# initialization of the flair model for creating embeddings of sentences
embeddings = DocumentPoolEmbeddings([WordEmbeddings('en')],pooling='mean',)

# chatbot class
class chatbot:

    # creating embeddings
    @staticmethod
    def prepare_embeddings(input_file,output_file):
        pass

    # processing user message and answering it
    @staticmethod
    def answer(message,embeddings_file,anwsers_file):
       pass

## Preparing Embeddings

Firstly we need to prepare a file containing our intents and their examples. This is a json dictionary that uses intents as keys and tables of examples as values.


In [ ]:
intents = {
  "hello": [
  "Hi",
  "Hello",
  "Welcome",
  "Good morning"
  ],
  "bye": [
    "Bye",
    "Later",
  
  ],
  "whatsup": [
    "How are you?",
    "What's up?",
  ],
  "about": [
    "Tell me about yourself",
    "Who are you?",
  ]
}

## Function that Constructs Embeddings for the Examples

In [ ]:
@staticmethod
    def prepare_embeddings(input_file,output_file):
        global embeddings

        # Creating new python dictionary for the embeddings
        embedded_intent_dict = {}

        # Opening the input file and loading it to python dictionary
        with open(input_file) as file:
            intent_dict = json.load(file)

        # For each intent we create a table in the embeddings dictionary
        for intent,examples in tqdm(intent_dict.items()):
            embedded_intent_dict[intent] = []

            # For each example in the intent, we create a Flair sentence object 
            # that we can later embed using the model specified earlier. 
            # Finally we add the embedded sentence to the table
            for example in examples:
                sentence = Sentence(example)
                embeddings.embed(sentence)
                embedded_intent_dict[intent].append(sentence.embedding.detach().numpy())

        # If the file doesn’t exist, we create it
        if not os.path.exists(os.path.dirname(output_file)):
            os.makedirs(os.path.dirname(output_file))

        # We save the embedded dict. We use pickle instead of json to store the numpy arrays
        pickle.dump(embedded_intent_dict,open( output_file, "wb+"))

## Answering the message

In [ ]:
ans = {
  "hello": [
    "Hello, what can I help you with?",
    "Hi what can I do for you today?"
  ],
  "bye": [
    "See you later",
    "See you next time"
 
  ],
  "whatsup": [
    "I feel happy answering your questions"
  ],
  "about": [
    "I am bot created in 50 lines of code"
  ]
}

In [ ]:
    @staticmethod
    def answer(message,embeddings_file,anwsers_file):
        global embeddings
        with open(embeddings_file, 'rb') as file:
            embedded_dict = pickle.load(file)
        message_sentence = Sentence(message)
        embeddings.embed(message_sentence)
        message_vector = message_sentence.embedding.detach().numpy()
        best_intent = ""
        best_score = 1
        for intent, examples in embedded_dict.items():
            for example in examples:
                score = cosine(message_vector, example)
                if(score<best_score):
                    best_score = score
                    best_intent = intent
        with open(anwsers_file) as file:
            anwsers_dict = json.load(file)
        if(best_intent in anwsers_dict):
            return random.choice(anwsers_dict[best_intent])
        else:
            return "Error intent not in dict"

## DRIVER Code

In [ ]:
if __name__ == "__main__":

    while True:
        input_message = input("Message: ")
        print(f"Bot:{chatbot.answer(input_message,embeddings_file='embedded_intents/test1.pkl',anwsers_file='answers/test1.json')}")
